IMPORTAMOS LIBRERIAS

In [1]:

!pip install geopy
!pip install scipy 



import math
import pandas as pd
import numpy as np
from scipy.stats import norm
import scipy.stats
import seaborn as sbs
import matplotlib.pyplot as plt
import geopy.distance

In [2]:

#TAXI
#armamos una funcion para calcular el tamaño de muestra
#asumiendo una distribucion normal
def sample_size(population_size, margin_of_error, confidence_level, sigma):
    z = norm.ppf(1 - (1 - confidence_level) / 2)
    n_0 = (z ** 2 * sigma ** 2) / (margin_of_error ** 2)
    n = n_0 / (1 + ((n_0 - 1) / population_size))
    return int(round(n))

#parametros 
population_size_taxi = 139000000
margin_of_error = 0.01
confidence_level = 0.99
#no conocemos el desvio estandard, por lo que asumimos maxima variabilidad, lo que implica un sigma de 0.5
sigma = 0.5

#pasamos parametros como argumentos a la funcion e imprimimos el tamaño de muestra
n_taxi = sample_size(population_size_taxi, margin_of_error, confidence_level, sigma)
print(f'Tamaño de muestra: {n_taxi}')

Tamaño de muestra: 16585


In [3]:
#comprobamos asumiendo un distribucion t de student con los mismo parametros el tamaño de la muestra
t_alpha_2=scipy.stats.t.ppf(1-0.01/2, df=population_size_taxi-1)
s=0.5
E=0.01
ty = (((t_alpha_2)**2)*(s**2))
n_taxi2=ty/(E**2)
n_taxi2
#la funcion devuelve una dimension muy similar para el dataset de taxis

16587.241958099352

In [4]:
population_size_bici = 13800000

#pasamos parametros como argumentos a la funcion e imprimimos el tamaño de muestra
n_bici = sample_size(population_size_bici, margin_of_error, confidence_level, sigma)
print(f'Tamaño de muestra: {n_bici}')

Tamaño de muestra: 16567


In [5]:
#comprobamos asumiendo un distribucion t de student con los mismo parametros el tamaño de la muestra
t_alpha_2=scipy.stats.t.ppf(1-0.01/2, df=population_size_bici-1)
s=0.5
E=0.01
ty = (((t_alpha_2)**2)*(s**2))
n_bici2=ty/(E**2)
n_bici2
#la funcion devuelve una dimension muy similar para el dataset de taxis

16587.246091027966

In [6]:
#el porcentaje de la muestra deseada es el tamaño de muestra dividido la poblacion
#n/139000000

In [7]:
#muestra en relacion a la poblacion de taxi
muestra_per_taxi=(n_taxi/population_size_taxi)
muestra_per_taxi

0.00011931654676258992

In [8]:
#muestra en relacion a la poblacion de bici
muestra_per_bici=(n_bici/population_size_bici)
muestra_per_bici

0.0012005072463768115

CREAMOS UN DATAFRAME CONSOLIDADO CON LOS ARCHIVOS DESCARGADOS

In [9]:
#TAXI
#creamos dataframe vacio
df_taxi=pd.DataFrame()
#bucle para cada 1 de los 12 archivos descargados, donde solo varia el mes en el nombre
for i in range(1,13):
    #lectura del archivo
    df_aux = pd.read_parquet(f'yellow_tripdata_2016-{i:02d}.parquet')
    #muestra aleatoria del 1%, con una semilla para reproducibilidad
    df_aux = df_aux.sample(frac=muestra_per_taxi, random_state=4)
    #concatenamos o adjuntamos el archivo muestreado en el consolidado
    df_taxi = pd.concat([df_taxi, df_aux]).reset_index(drop=True)    

FileNotFoundError: [Errno 2] No such file or directory: 'yellow_tripdata_2016-01.parquet'

PROCEDEMOS A AGREGAR ALGUNAS COLUMNAS EN LOS DOS DATAFRAMES

In [ ]:
#BICI
#creamos dataframe vacio
df_bici=pd.DataFrame()
#bucle para cada 1 de los 12 archivos descargados, donde solo varia el mes en el nombre
for i in range(1,13):
    #lectura del archivo
    df_aux = pd.read_csv(f'Bicis/2016{i:02d}-citibike-tripdata.csv')
    #muestra aleatoria del 1%, con una semilla para reproducibilidad
    df_aux = df_aux.sample(frac=muestra_per_bici, random_state=4)
    df_aux.columns=df_aux.columns.str.lower()
        # verificar si la columna 'startime' está presente en df_aux
    if 'starttime' in df_aux.columns:
        df_aux = df_aux.rename(columns={'starttime': 'start time'})
    if 'stoptime' in df_aux.columns:
        df_aux = df_aux.rename(columns={'stoptime': 'stop time'})
    if 'tripduration' in df_aux.columns:
        df_aux = df_aux.rename(columns={'tripduration': 'trip duration'})
    if 'bikeid' in df_aux.columns:
        df_aux = df_aux.rename(columns={'bikeid': 'bike id'})
    if 'usertype' in df_aux.columns:
        df_aux = df_aux.rename(columns={'usertype': 'user type'})
    #concatenamos o adjuntamos el archivo muestreado en el consolidado
    df_bici = pd.concat([df_bici, df_aux]).reset_index(drop=True)

In [12]:
df_bici=pd.read_csv('Bicis.csv')

In [13]:
df_taxi.describe()

ValueError: Cannot describe a DataFrame without columns

In [ ]:
#TAXI
#agregamos algunas columnas utiles: duracion del viaje en minutos, dias de semana y el mes.

# Convierte las columnas de fecha y hora en objetos de fecha y hora de Pandas.
df_taxi['tpep_pickup_datetime'] = pd.to_datetime(df_taxi['tpep_pickup_datetime'])
df_taxi['tpep_dropoff_datetime'] = pd.to_datetime(df_taxi['tpep_dropoff_datetime'])

# Crea una nueva columna de duración de viaje en minutos
df_taxi['trip_duration_mins'] = (df_taxi['tpep_dropoff_datetime'] - df_taxi['tpep_pickup_datetime']).dt.total_seconds() / 60

In [ ]:
#dias de semana
df_taxi['week_day']=df_taxi['tpep_pickup_datetime'].dt.dayofweek
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df_taxi['week_day']=df_taxi['week_day'].map(day_names)

In [ ]:
#mes del año
df_taxi['month'] = pd.DatetimeIndex(df_taxi['tpep_pickup_datetime']).month

In [14]:
#BICI
# Agregamos una columna con el día de la semana que se corresponde con el viaje
df_bici['stop time']=pd.to_datetime(df_bici['stop time'])
df_bici['week_day']=df_bici['stop time'].dt.dayofweek
day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df_bici['week_day'] = df_bici['week_day'].map(day_names)

In [15]:
df_bici['month'] = pd.DatetimeIndex(df_bici['stop time']).month

In [16]:
# Agregamos una columna con las distancias recorridas en cada viaje

# Definir una función para calcular la distancia entre dos puntos
df_bici_copy=df_bici.copy()

def distancia_entre_puntos(row):
    coords_1 = (row['start station latitude'], row['start station longitude'])
    coords_2 = (row['end station latitude'], row['end station longitude'])
    return geopy.distance.distance(coords_1, coords_2).miles

# Aplicar la función a cada fila del dataframe y calcular la distancia recorrida en cada viaje
df_bici_copy.loc[:,'distancia (ml)'] = df_bici_copy.apply(distancia_entre_puntos, axis=1)
df_bici=df_bici_copy

In [17]:
#duracion del viaje en minutos
df_bici['trip duration mins']=df_bici['trip duration']/60

In [18]:
df_bici.tail()

,Unnamed: 0,trip duration,start time,stop time,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bike id,user type,birth year,gender,week_day,month,distancia (ml),trip duration mins
16615,16615,1279,2016-12-12 17:18:33,2016-12-12 17:39:53,383,Greenwich Ave & Charles St,40.735238,-74.000271,523,W 38 St & 8 Ave,40.754666,-73.991382,20528,Subscriber,1964.0,2,Monday,12,1.419433,21.316667
16616,16616,1725,2016-12-30 19:38:45,2016-12-30 20:07:31,147,Greenwich St & Warren St,40.715422,-74.011220,488,W 39 St & 9 Ave,40.756458,-73.993722,22301,Subscriber,1969.0,1,Friday,12,2.976819,28.750000
16617,16617,937,2016-12-21 09:31:19,2016-12-21 09:46:57,498,Broadway & W 32 St,40.748549,-73.988084,151,Cleveland Pl & Spring St,40.722104,-73.997249,17979,Subscriber,1982.0,2,Wednesday,12,1.887101,15.616667
16618,16618,412,2016-12-11 16:13:01,2016-12-11 16:19:53,248,Laight St & Hudson St,40.721854,-74.007718,368,Carmine St & 6 Ave,40.730386,-74.002150,19105,Subscriber,1978.0,1,Sunday,12,0.657305,6.866667
16619,16619,1267,2016-12-28 04:23:37,2016-12-28 04:44:44,3263,Cooper Square & E 7 St,40.729236,-73.990868,380,W 4 St & 7 Ave S,40.734011,-74.002939,26395,Customer,NaN,0,Wednesday,12,0.714149,21.116667


Iniciamos la limpieza del dataset, en este caso tratando de identificar si hay valores negativos.

In [19]:
#TAXI
#comenzamos por los nulls
columns_with_nulls = df_taxi.columns[df_taxi.isnull().any()].tolist()
print(columns_with_nulls)

[]


In [20]:
#la totalidad de los valores de las dos columnas con nulos, por lo que no vamos a usar las columnas y no consideramos relevante eliminarlas.

In [21]:
df_taxi.describe()

ValueError: Cannot describe a DataFrame without columns

In [ ]:
#consultamos viajes con cero pasajeros
pasajeros_cero=df_taxi.query('passenger_count==0')

In [ ]:
#creamos una lista con los indices de las filas con cero pasajeros
index_list1 = pasajeros_cero.index.tolist()
#eliminamos esas filas de df_taxi
df_taxi = df_taxi.drop(index_list1, axis=0)

In [ ]:
#consultamos viajes distancia de viaje 0
distancia_cero=df_taxi.query('trip_distance==0')
#creamos una lista con los indices de las filas con distancia cero.
index_list1 = distancia_cero.index.tolist()
#eliminamos esas filas de df_taxi
df_taxi = df_taxi.drop(index_list1, axis=0)

In [ ]:
# seleccionamos solo columnas de tipo numerico
numeric_cols = df_taxi.select_dtypes(include=['number'])
# seleccionamos solo filas con algun valor negativo.
neg_rows = df_taxi[numeric_cols.lt(0).any(axis=1)]
neg_rows

In [ ]:
#creamos una lista con los indices de las filas con valores negativas
index_list = neg_rows.index.tolist()
#eliminamos esas filas de df_taxi
df_taxi = df_taxi.drop(index_list, axis=0)

In [ ]:
#BICI
#chequeamos viajes con distancia en 0 
df_bici_d0=df_bici.loc[df_bici['distancia (ml)'] == 0]

In [ ]:
#eliminamos las filas del dataset, ya que distorsionan el analisis
df_bici = df_bici.drop(df_bici[df_bici['distancia (ml)'] == 0].index)

ELIMINAMOS FILAS CON VALORES 0 EN COORDENADAS

In [22]:
# eliminar las filas con valores nulos y/o con valores reales 0.0 en las columnas "start station latitude", "start station longitude", "end station latitude" y "end station longitude"
df_bici = df_bici.loc[(df_bici['start station latitude'] != 0.0) & 
                      (df_bici['start station longitude'] != 0.0) & 
                      (df_bici['end station latitude'] != 0.0) & 
                      (df_bici['end station longitude'] != 0.0) & 
                      (~df_bici['start station latitude'].isnull()) & 
                      (~df_bici['start station longitude'].isnull()) & 
                      (~df_bici['end station latitude'].isnull()) & 
                      (~df_bici['end station longitude'].isnull())]

In [23]:
#Taxi
#chequamos si hay duplicados
hay_duplicados = df_taxi.duplicated().any()

if hay_duplicados:
    print ( "Hay")
else:
    print ("No Hay")

No Hay


In [24]:
#Bici
#chequamos si hay duplicados
hay_duplicados = df_bici.duplicated().any()

if hay_duplicados:
    print ( "Hay")
else:
    print ("No Hay")

No Hay


In [25]:
#Bici
# seleccionamos solo columnas de tipo numerico
numeric_cols = df_bici.select_dtypes(include=['number'])
sd=numeric_cols.drop(['start station longitude','end station longitude'],axis=1)
# seleccionamos solo filas con algun valor negativo.
neg_rows = df_bici[sd.lt(0).any(axis=1)]
neg_rows

,Unnamed: 0,trip duration,start time,stop time,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bike id,user type,birth year,gender,week_day,month,distancia (ml),trip duration mins
